# Decorators

Dirk Lüdtke (bizzarq software)  
2025-04-30 Python Barcelona Meetup

<img alt="Christmas decoration inside a data center" src="img/1_decoration.png" width="75%">

## Introduction

### Examples

- @classmethod
- @app.route (Flask)
- @validator (Pydantic)
- @tf.function (TensorFlow)

### Origin

- available since Nov. 2004 (Python 2.4)
- Java compiler annotation @Deprecated released Sep. 2004 (Java 1.5)
  - Java docstring @deprecated since 1996? (Java 1.0?)
- Objective-C \[Attribute\] before 2000

### Others
- @syntax: Grovy, Scala, Kotlin, Swift, Dart, TypeScript, Javascript (experimental)
- \#\[syntax\]: Rust, PHP

---
## Functional Programming

<img alt="Python looking at a blackboard with formulas" src="img/2_functions.png" width="75%">

### Functions as Function Arguments

In [ ]:
def function1a():
    print('function1a started and finished')

def function1b():
    print('function1b started and finished')

def function2(fun):
    print('function2 started')
    fun()
    print('function2 finished')

function2(function1a)
function2(function1b)

### Functions as Return Values

In [ ]:
def function1(opp):
    def function2(a, b):
        if opp == "*":
            return a * b
        return a + b
    return function2

mul = function1('*')
add = function1('+')

print('mul(3, 4) =', mul(3, 4))
print('add(3, 4) =', add(3, 4))

---
## Howto Decorators

### @simple_decorator

- function X
- which receives function Y and
- returns function Z which is called instead of function Y

**Example: Logging**

In [ ]:
def log_function(fun):

    def decorate(*args, **kwargs):
        print(f'calling "{fun.__name__}" with {args} and {kwargs}')
        result = fun(*args, **kwargs)
        print(f'"{fun.__name__}" returned {result}')
        return result

    return decorate

@log_function
def add(x, y):
    return x + y

print(add(1, 1))
print(add(y=3, x=2))

### @decorator_with_args(*args, **kwargs)

- function W
- which receives the desired arguments and
- returns a simple decorator
  - function X
  - which receives function Y and
  - returns function Z which is called instead of function Y

**Example: Repeat**

In [ ]:
def repeat(n: int):
    def create_decorator(fun):
        def decorate(value):
            for _ in range(n):
                value = fun(value)
            return value
        return decorate
    return create_decorator

@repeat(5)
def plus_one(n):
    return n + 1
print('plus one repeated 5 times:', plus_one(0))

### Combined Decorators

**Example: Nested Repeat**

In [ ]:
@repeat(5)
@repeat(5)
@repeat(5)
def plus_one_b(n):
    return n + 1

print('plus one repeated 5 * 5 * 5 times:', plus_one_b(0))

**Example: Nested Decoration**

In [ ]:
def repeat2(n: int, m: int):
    @repeat(m)
    def create_decorator(fun):
        @repeat(n)
        def decorate(value):
            return fun(value)
        return decorate
    return create_decorator

@repeat2(5, 4)
def plus_one_c(n):
    return n + 1

print('plus one repeated 5 ** 4 times:', plus_one_c(0))

<img alt="Christmas tree balls in infinite grid" src="img/3_repeat.png" width="75%">

---
## Algorithms

### Divide and Conquer

In [ ]:
def divide_and_conquer(fun):
    def decorate(elements):
        if (len(elements) < 2):
            return elements
        mid = len(elements) // 2
        return fun(decorate(elements[:mid]), decorate(elements[mid:]))
    return decorate


<img alt="Santa as a conqueror" src="img/4_conquer.png" width="75%">

**Example: Mergesort**

In [ ]:
@divide_and_conquer
def merge_lists(list1, list2):
    result = []
    pos1, pos2 = 0, 0
    while pos1 < len(list1) or pos2 < len(list2):
        if pos2 >= len(list2) or (pos1 < len(list1) and list1[pos1] < list2[pos2]):
            result.append(list1[pos1])
            pos1 += 1
        else:
            result.append(list2[pos2])
            pos2 += 1
    return result

print(merge_lists([9, 4, 3, 8, 6, 5, 2, 7, 1]))

**Example: Revert**

In [ ]:

@divide_and_conquer
def reorder_lists(list1, list2):
    return list2 + list1

print(reorder_lists([9, 4, 3, 8, 6, 5, 2, 7, 1]))


### Bounded Series of Numbers

In [ ]:
def is_bounded(start, bound_function, max_steps):
    def create_decorator(fun):
        def decorate(c):
            value = start
            for _ in range(max_steps):
                value = fun(value, c)
                if not bound_function(value):
                    return False
            return True
        return decorate
    return create_decorator

**Example: Mandelbrot Set**

<img alt="Mandelbrot set in color" src="img/5_mandelbrot.png" width="75%"/>

In [ ]:

@is_bounded(0, lambda z: abs(z) < 2, 1000)
def mandelbrot_step(z, c):
    return z ** 2 + c

symbols = {True: '*', False: ' '}
for row in range(21):
    i = (row - 10) / 10
    for col in range(61):
        r = (col - 40) / 20
        symbol = symbols[mandelbrot_step(complex(r, i))]
        print(symbol, end='')
    print()

---
## Evaluation

### Wrapping Functions

**with decorator**

In [ ]:
def wrap_decorator(function):
    def wrap_decorate(*args, **kwargs):
        print(f'wrap_decorate {function.__name__} {args} {kwargs}')
        return function(*args, **kwargs)
    return wrap_decorate

@wrap_decorator
def wrap_decorated(greeting):
    print(f'{greeting} wrap_decorated')

wrap_decorated('Hello')

**old school**

In [ ]:
def wrap_old_school(function, *args, **kwargs):
    print(f'wrap_old_school {function.__name__} {args} {kwargs}')
    return function(*args, **kwargs)

def old_school(greeting):
    print(f'{greeting} old_school')

wrap_old_school(old_school, 'Hello')

### Storing Functions

**with decorator**

In [ ]:
decorator_store = {}
def store_decorator(key):
    def create_decorator(function):
        decorator_store[key] = function
        return function
    return create_decorator

@store_decorator('my_function')
def store_decorated(greeting):
    print(f'{greeting} decorator_registered')

print(decorator_store)

**old school**

In [ ]:
old_school_store = {}
def store_old_school(key, function):
    old_school_store[key] = function

store_old_school('my_function', old_school)
print(old_school_store)

### Wrapping and Storing

**with decorator**

In [ ]:
@wrap_decorator
@store_decorator('my_function')
def wrap_and_store_decorated(greeting):
    print(f'{greeting} wrap_and_store_decorated')

wrap_and_store_decorated('Good bye')
print(decorator_store)

In [ ]:
wrap_old_school(old_school, 'Good bye')
store_old_school('my_function', old_school)
print(old_school_store)

### Function Name Changes

In [ ]:
def change_name(function):
    def internal_name(*args, **kwargs):
        return function(*args, **kwargs)
    return internal_name

@change_name
def external_name():
    pass

print(external_name.__name__)

In [ ]:
import functools

def dont_change_name(function):
    @functools.wraps(function)
    def internal_name(*args, **kwargs):
        return function(*args, **kwargs)
    return internal_name

@dont_change_name
def external_name():
    pass

print(external_name.__name__)

### Misuse

1. Evil library

In [ ]:
def evil_decorator(function):
    def internal_name(*args, **kwargs):
        print('I am extremely evil')
    return internal_name

2. Trusted function

In [ ]:
@evil_decorator
def trusted_function():
    print('I am harmless')

3. Unsuspicious use

In [ ]:
trusted_function()

---
## Material

Python PEP
: https://peps.python.org/pep-0318/

Tutorial
: https://realpython.com/primer-on-python-decorators/

Functools
: https://docs.python.org/3/library/functools.html
  - @functools.cache (cache results of previous calls)
  - @functools.wraps (decorates a decorator function for looking like the decorated function)

Google style guide
: https://google.github.io/styleguide/pyguide.html
  - 2.17.2 Pros
  : Elegantly specifies some transformation on a method; the transformation might eliminate some repetitive code, enforce invariants, etc.
  - 2.17.3 Cons
  : Decorators can perform arbitrary operations on a function’s arguments or return values, resulting in surprising implicit behavior. Additionally, decorators execute at object definition time. For module-level objects (classes, module functions, …) this happens at import time. Failures in decorator code are pretty much impossible to recover from.

---
## Conclusion

<img alt="desk after Christmas party" src="img/6_conclusion.png" width="75%">

bird-view
- more boilerplate code
- functions cannot be re-used in different contexts
- implicit behavior
- debugging a bit tedious

creator of decorator
- more control
- good extensibility

user of decorator
- more convenience
- hidden complexity

--> good for libraries / frameworks  
--> no use case for own projects

---
## Thank You!